# Spikeinterface Cookbook


In [1]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Playing with Recording and Sorting objects

#### Generate synthetic data

In [2]:
from spikeinterface.core import generate_recording 

recording = generate_recording(num_channels=3, durations=[10])
recording

SyntheticRecording: 3 channels - 30.0kHz - 1 segments - 300,000 samples - 10.00s - float32 dtype 
                    3.43 MiB

In [3]:
from spikeinterface.core import generate_sorting 

sorting = generate_sorting(num_units=3, durations=[10])
sorting

NumpySorting: 3 units - 1 segments - 30.0kHz

#### Setting channel names and unit IDs

In [4]:
recording = recording.rename_channels(new_channel_ids=["a", "b", "c"])  # This is not in-place
recording.get_channel_ids()

array(['a', 'b', 'c'], dtype='<U1')

In [5]:
sorting = sorting.rename_units(new_unit_ids=["unit1", "unit2", "unit3"])  # This is not in-place
sorting.get_unit_ids()

array(['unit1', 'unit2', 'unit3'], dtype='<U5')

### Slicing a Recording and Sorting Objects

#### Channels

In [6]:
channel_sliced_recording = recording.select_channels(channel_ids=["a", "b"])
channel_sliced_recording

SyntheticRecording: 2 channels - 30.0kHz - 1 segments - 300,000 samples - 10.00s - float32 dtype 
                    2.29 MiB

#### Units

In [7]:
unit_selected_sorting = sorting.select_units(unit_ids=["unit1", "unit2"])
unit_selected_sorting

UnitsSelectionSorting: 2 units - 1 segments - 30.0kHz

### Frames / Time

In [8]:
sliced_recording = recording.frame_slice(start_frame=0, end_frame=1000)
sliced_recording

SyntheticRecording: 3 channels - 30.0kHz - 1 segments - 1,000 samples - 0.03s (33.33 ms) 
                    float32 dtype - 11.72 KiB

In [9]:
sliced_sorting = sorting.frame_slice(start_frame=0, end_frame=1000)
sliced_sorting

FrameSliceSorting: 3 units - 1 segments - 30.0kHz

### Combining recordings

#### Concatenating recordings (across time)

In [10]:
from spikeinterface.core import concatenate_recordings

recording1 = generate_recording(num_channels=3, durations=[10])
recording2 = generate_recording(num_channels=3, durations=[10])

concanted_recordings = concatenate_recordings([recording1, recording2])

assert concanted_recordings.get_duration() == recording1.get_duration()  + recording2.get_duration()


#### Aggregating channels to a single recording

In [11]:
from spikeinterface.core import aggregate_channels

recording1 = generate_recording(num_channels=3, durations=[10], set_probe=False)  # To avoid location check
recording1 = recording1.rename_channels(new_channel_ids=["a", "b", "c"])
recording2 = generate_recording(num_channels=2, durations=[10], set_probe=False)  
recording2 = recording2.rename_channels(new_channel_ids=["d", "e"])

aggregated_recording = aggregate_channels([recording1, recording2])  
assert aggregated_recording.get_num_channels() == 5
assert list(aggregated_recording.get_channel_ids()) == ['a', 'b', 'c', 'd', 'e']  # Failing right now

### Summary of operations




![Recording Operations](./recording_operations.png)


## Parallel Processing

### What do we parallelize over

![Chuking Description](./parallel_processing.png)

### Parameters to control paralell execution 

* lenght of the chunk
    * chunk_duration :  Lenght of the chunk in seconds
    * chunk_size: Number of samples per chunk
    * chunk_memory: Memory usage for each job
    * total_memory Total memory usage 
* n_jobs: Number of jobs to use. With -1 the number of jobs is the same as number of cores
* progress_bar: Whether to show a progress bar
* mp_context: fork, span or forkserver

In [12]:
from spikeinterface.core import generate_recording, write_binary_recording
import tempfile

recording = generate_recording(num_channels=3, durations=[10])


job_kwargs = {"n_jobs":2, "chunk_duration": 5.0, "progress_bar": 1, 'progress_bar': True}


with tempfile.NamedTemporaryFile(suffix='.raw', delete=False) as temp_file:
    temporary_file_path = temp_file.name
    write_binary_recording(recording=recording, file_paths=[temporary_file_path], **job_kwargs)



write_binary_recording:   0%|          | 0/2 [00:00<?, ?it/s]

![job_kwargs](./job_kwargs.png)

### Setting global job_kwargs

In [13]:
from spikeinterface import set_global_job_kwargs, get_global_job_kwargs
from spikeinterface.core import generate_recording, write_binary_recording
import tempfile

recording = generate_recording(num_channels=3, durations=[10])


print(get_global_job_kwargs())

with tempfile.NamedTemporaryFile(suffix='.raw', delete=False) as temp_file:
    temporary_file_path = temp_file.name
    write_binary_recording(recording=recording, file_paths=[temporary_file_path])


set_global_job_kwargs(n_jobs=2, chunk_duration=5.0, progress_bar=True)

with tempfile.NamedTemporaryFile(suffix='.raw', delete=False) as temp_file:
    temporary_file_path = temp_file.name
    write_binary_recording(recording=recording, file_paths=[temporary_file_path])




{'n_jobs': 1, 'chunk_duration': '1s', 'progress_bar': True, 'mp_context': None, 'max_threads_per_process': 1}


/home/samuel/Documents/SpikeInterface/spikeinterface/src/spikeinterface/core/job_tools.py:103: UserWarning: `n_jobs` is not set so parallel processing is disabled! To speed up computations, it is recommended to set n_jobs either globally (with the `spikeinterface.set_global_job_kwargs()` function) or locally (with the `n_jobs` argument). Use `spikeinterface.set_global_job_kwargs?` for more information about job_kwargs.
  warnings.warn(


write_binary_recording:   0%|          | 0/10 [00:00<?, ?it/s]

write_binary_recording:   0%|          | 0/2 [00:00<?, ?it/s]

Stop to mentioned cautionary tales:
* Performance is highly dependent on the data, operations and the hardware.
* The best way to optimize is to try different configurations and see what works best for your data and hardware.
* Threading vs multiprocessing: Threading is generally faster for I/O bound tasks, while multiprocessing is better for CPU bound tasks.
* Anything to add?

## Saving Recording and Sorting Objects

### Binary format

#### Recording

In [14]:
from pathlib import Path
from spikeinterface.core import generate_recording

recording = generate_recording(num_channels=3, durations=[10], set_probe=True)
recording = recording.rename_channels(new_channel_ids=["a", "b", "c"])  # This is not in-place
recording.set_property("a_property", ["value1", "value2", "value3"])  # This is in place



job_kwargs={'progress_bar': True, "verbose":True, "n_jobs":2}

folder_path = Path("./test_recording")


binary_recording = recording.save_to_folder(folder=folder_path,  overwrite=True, **job_kwargs)   
binary_recording

write_binary_recording 
n_jobs=2 - samples_per_chunk=150,000 - chunk_memory=1.72 MiB - total_memory=3.43 MiB - chunk_duration=5.00s


write_binary_recording:   0%|          | 0/2 [00:00<?, ?it/s]

SyntheticRecording: 3 channels - 30.0kHz - 1 segments - 300,000 samples - 10.00s - float32 dtype 
                    3.43 MiB

#### Sorting

In [15]:
from pathlib import Path
from spikeinterface.core import generate_sorting

sorting = generate_sorting(num_units=3, durations=[10])
sorting = sorting.rename_units(new_unit_ids=["unit1", "unit2", "unit3"])  # This is not in-place

sorting.set_property("a_property", ["value1", "value2", "value3"])  # This is in place


folder_path = Path("./test_sorting")

job_kwargs={'progress_bar': True, "verbose":True, "n_jobs":2}
binary_sorting = sorting.save_to_folder(folder=folder_path,  overwrite=True, **job_kwargs)   
binary_sorting


NumpyFolder: 3 units - 1 segments - 30.0kHz

In [16]:
sorting

UnitsSelectionSorting: 3 units - 1 segments - 30.0kHz

### Zarr format

#### Recording

In [17]:
from pathlib import Path
from spikeinterface.core import generate_recording

recording = generate_recording(num_channels=3, durations=[10], set_probe=True)
recording = recording.rename_channels(new_channel_ids=["a", "b", "c"])  # This is not in-place
recording.set_property("a_property", ["value1", "value2", "value3"])  # This is in place

folder_path = Path("./test_recording.zarr")

job_kwargs={'progress_bar': True, "verbose":True, "n_jobs":2}
zarr_recording = recording.save_to_zarr(folder=folder_path,  overwrite=True, **job_kwargs)   
zarr_recording

write_zarr_recording 
n_jobs=2 - samples_per_chunk=150,000 - chunk_memory=1.72 MiB - total_memory=3.43 MiB - chunk_duration=5.00s


write_zarr_recording:   0%|          | 0/2 [00:00<?, ?it/s]

SyntheticRecording: 3 channels - 30.0kHz - 1 segments - 300,000 samples - 10.00s - float32 dtype 
                    3.43 MiB

In [19]:
zarr_recording._root.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, icon='table', name='channel_ids (3,) <U1'…

#### Sorting

In [20]:
from pathlib import Path
from spikeinterface.core import generate_sorting

sorting = generate_sorting(num_units=3, durations=[10])
sorting = sorting.rename_units(new_unit_ids=["unit1", "unit2", "unit3"])  # This is not in-place

sorting.set_property("a_property", ["value1", "value2", "value3"])  # This is in place


folder_path = Path("./test_sorting.zarr")

job_kwargs={'progress_bar': True, "verbose":True, "n_jobs":2}
zarr_sorting = sorting.save_to_zarr(folder=folder_path,  overwrite=True, **job_kwargs)   
zarr_sorting


ZarrSortingExtractor: 3 units - 1 segments - 30.0kHz

In [21]:
zarr_sorting._root.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, name='properties', nodes=(Node(disabled=T…

### Saving is portable

In [22]:
from pathlib import Path
from spikeinterface.core import generate_recording
from spikeinterface.core import load_extractor  

recording = generate_recording(num_channels=3, durations=[10], set_probe=True)


# Save a recording within a nested folder
base_folder = Path.cwd() / "saving_recording_and_moving" 
original_recording_folder = base_folder / "folderA" / "recording_folder"  

recording = recording.save_to_folder(folder=original_recording_folder, overwrite=True)


# We move the folder from its original location one level up to the current folder
another_folder = base_folder / "folderB"
another_folder.mkdir(exist_ok=True)
destination_folder = another_folder / "recording_folder"

original_recording_folder.rename(destination_folder)



load_extractor(file_or_folder_or_dict=destination_folder)

write_binary_recording 
n_jobs=2 - samples_per_chunk=150,000 - chunk_memory=1.72 MiB - total_memory=3.43 MiB - chunk_duration=5.00s


write_binary_recording:   0%|          | 0/2 [00:00<?, ?it/s]

SyntheticRecording: 3 channels - 30.0kHz - 1 segments - 300,000 samples - 10.00s - float32 dtype 
                    3.43 MiB

## Saving preprocessing provenance

In [23]:
from pathlib import Path

from spikeinterface.core import generate_recording, load_extractor
from spikeinterface.preprocessing import bandpass_filter, common_reference

# First we simulate having raw data (can be large!) in a folder 
simulated_recording = generate_recording(num_channels=3, durations=[10])
base_folder = Path.cwd() / "working_with_preprocessing_provenance"

raw_data_location = base_folder/ "raw_data_location_folder"
raw_data_location.mkdir(parents=True, exist_ok=True)
recording_saved = simulated_recording.save(folder=raw_data_location, overwrite=True)


# Now our common analysis pipeline starts by loading the raw data
raw_data_recording = load_extractor(file_or_folder_or_dict=raw_data_location)
# And then we apply some preprocessing steps
recording_preprocessed = bandpass_filter(common_reference(raw_data_recording))

# Note that we can save our preprocessed data for faster access afterwards:
# recording_preprocessed.save(folder=Path.cwd() / "preprocessed_data_folder")
# But maybe we only want to save our provenance data, that is, our pre-processing pipeline and parameters
# For this we can save oure preprocessing pipeline as a json file

# dump_to_json without relative_to

json_file_path = base_folder / "analysis_folder" / "preprocessed_pipeline.json"
recording_preprocessed.dump_to_json(file_path=json_file_path)


json_file_path_relative = base_folder / "analysis_folder" / "preprocessed_pipeline_relative.json"
recording_preprocessed.dump_to_json(file_path=json_file_path_relative, relative_to=base_folder)


# This then can be loaded again to recover the pipeline
recovered_pipeline = load_extractor(file_or_folder_or_dict=json_file_path)

# We move the json to a new folder
new_base_folder = Path.cwd() / "new_working_with_preprocessing_provenance"
new_base_folder.mkdir(exist_ok=True)

base_folder.rename(new_base_folder)

new_json_file_path = new_base_folder / "analysis_folder" / "preprocessed_pipeline.json"
assert new_json_file_path.is_file(), "The json file was not moved correctly" 

# Try to load it
try:
    recovered_pipeline = load_extractor(file_or_folder_or_dict=new_json_file_path)
except:
    print("This generated an error because the references to the raw data were absolute")
    
# We can solve this by saving the json file with relative paths

new_json_file_path_with_relative = new_base_folder / "analysis_folder" / "preprocessed_pipeline_relative.json"
assert new_json_file_path_with_relative.is_file(), "The json file with relative was not moved correctly"
recovered_pipeline = load_extractor(file_or_folder_or_dict=new_json_file_path_with_relative, base_folder=new_base_folder)
recovered_pipeline

write_binary_recording 
n_jobs=2 - samples_per_chunk=150,000 - chunk_memory=1.72 MiB - total_memory=3.43 MiB - chunk_duration=5.00s


write_binary_recording:   0%|          | 0/2 [00:00<?, ?it/s]

This generated an error because the references to the raw data were absolute


SyntheticRecording: 3 channels - 30.0kHz - 1 segments - 300,000 samples - 10.00s - float32 dtype 
                    3.43 MiB